In [1]:
# TODO : CTC Loss
from typing import List

import cv2
import gdown
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda

#import tensorflow as tf

In [2]:
# !jupyter nbconvert lip_keypoint.ipynb --to python

[NbConvertApp] Converting notebook lip_keypoint.ipynb to python
[NbConvertApp] Writing 9265 bytes to lip_keypoint.py


In [1]:
df3 = pd.read_csv("face_landmarks_data3.csv", index_col=0)
df4 = pd.read_csv("face_landmarks_data4.csv", index_col=0)
df5 = pd.read_csv("face_landmarks_data5.csv", index_col=0)
df = pd.concat([df3, df4, df5], ignore_index=True)

NameError: name 'pd' is not defined

In [3]:
video_path_counts = df['video_path'].value_counts()

video_paths_to_keep = video_path_counts[video_path_counts == 3000].index

df = df[df['video_path'].isin(video_paths_to_keep)]

In [4]:
mean_x = np.mean(df['x'])
std_x = np.std(df['x'])
mean_y = np.mean(df['y'])
std_y = np.std(df['y'])
mean_z = np.mean(df['z'])
std_z = np.std(df['z'])

# df['x'] = (df['x'] - mean_x) / std_x
# df['y'] = (df['y'] - mean_y) / std_y
# df['z'] = (df['z'] - mean_z) / std_z

In [62]:
# df = df[:4000]

In [5]:
df.head(5)

,frame,x,y,z,visibility,video_path
0,1,0.528295,0.712854,-0.031539,0.0,data3/s3\bbaf1s.mpg
1,1,0.543726,0.710494,-0.030296,0.0,data3/s3\bbaf1s.mpg
2,1,0.559562,0.715571,-0.022568,0.0,data3/s3\bbaf1s.mpg
3,1,0.570321,0.721982,-0.012750,0.0,data3/s3\bbaf1s.mpg
4,1,0.528245,0.728252,-0.018017,0.0,data3/s3\bbaf1s.mpg


In [4]:
df.groupby('video_path').count()

,frame,x,y,z,visibility
video_path,,,,,
data3/s3\bbaf1s.mpg,3000,3000,3000,3000,3000
data3/s3\bbaf2p.mpg,3000,3000,3000,3000,3000
data3/s3\bbaf3a.mpg,3000,3000,3000,3000,3000
data3/s3\bbafzn.mpg,3000,3000,3000,3000,3000
data3/s3\bbal4n.mpg,3000,3000,3000,3000,3000
...,...,...,...,...,...
data5/s5\swwpzp.mpg,3000,3000,3000,3000,3000
data5/s5\swwv2n.mpg,3000,3000,3000,3000,3000
data5/s5\swwv3s.mpg,3000,3000,3000,3000,3000


In [5]:
df_filtered = df.copy()
chunks = []
num_frames_per_chunk = 3000
for video_path, group in df_filtered.groupby('video_path'):
    num_frames = len(group)
    num_chunks = num_frames // num_frames_per_chunk
    for i in range(num_chunks):
        chunk = group.iloc[i*num_frames_per_chunk:(i+1)*num_frames_per_chunk]
        chunk_reshaped = chunk[['x', 'y', 'z']].values.reshape(-1, 75, 40*3)
        chunks.append(chunk_reshaped)

input_data = np.concatenate(chunks, axis=0)

In [6]:
input_data.shape

(2977, 75, 120)

In [7]:
def create_vocab():
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab


def char_to_int(char):
    # shift 1 
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab.index(char) + 1 if char in vocab else -1

def int_to_char(index):
    # shift 1 
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab[index - 1] if 1 <= index <= len(vocab) else ''

def load_alignments(path:str) -> List[str]:
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            tokens.extend([*line[2]])
            tokens.append(' ')
    return [char_to_int(token) for token in tokens]

all_alignments = []
for video_path in df['video_path'].unique():
    datapath = video_path.split('/')[0]
    speaker_path = video_path.split('/')[-1].split('\\')[0]
    vid_path = video_path.split('/')[-1].split('\\')[-1].split('.')[0]
    
    alignment_path = os.path.join(f'{datapath}','align',f'{vid_path}.align')
    alignments = load_alignments(alignment_path) 
    all_alignments.append(alignments)


In [8]:
len(all_alignments)

2977

In [11]:
# video_path = df['video_path'].iloc[0]
# datapath = video_path.split('/')
# speakert_path = video_path.split('/')[-1].split('\\')[0]
# vid_path = video_path.split('/')[-1].split('\\')[-1].split('.')[0]

In [9]:

label_data = [np.array(label) for label in all_alignments if label]
# Example of shaping your input data and label data for model training
#input_data = np.stack(chunks)  # Ensure that chunks are correctly reshaped and stacked

# Check shapes (important for debugging)
print("Input data shape:", input_data.shape)
print("Number of label sequences:", len(label_data))



Input data shape: (2977, 75, 120)
Number of label sequences: 2977


In [11]:
input_data.shape

(2977, 75, 120)

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
dataset = TensorDataset(input_data, label_data)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
class LipReadingRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LipReadingRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.input_layer = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.LayerNorm(hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.LayerNorm(hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.LayerNorm(hidden_size)
        )
        
        self.bilstm = nn.LSTM(hidden_size, hidden_size, num_layers=3, batch_first=True, bidirectional=True, dropout=0.3)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for layer in self.input_layer:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
        
        for name, param in self.bilstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
                
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x):
        x = self.input_layer(x)  
        h0 = torch.zeros(6, x.size(0), self.hidden_size).to(x.device)  # 6 for 3 layers bidirectional
        c0 = torch.zeros(6, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.bilstm(x, (h0, c0))
        out = self.dropout(out)
        out = self.fc(out)  
        
        return out


In [21]:
first_batch = next(iter(train_loader))
first_input, _, labels, _ = first_batch 
labels

tensor([[ 2,  9, 14, 36, 23,  8,  9, 20,  5, 36,  1, 20, 36, 19, 36, 14,  9, 14,
          5, 36, 19, 15, 15, 14, 36,  0,  0,  0,  0,  0,  0],
        [16, 12,  1,  3,  5, 36, 23,  8,  9, 20,  5, 36, 23,  9, 20,  8, 36, 24,
         36,  5,  9,  7,  8, 20, 36, 14, 15, 23, 36,  0,  0],
        [19,  5, 20, 36,  7, 18,  5,  5, 14, 36,  1, 20, 36,  2, 36, 14,  9, 14,
          5, 36, 14, 15, 23, 36,  0,  0,  0,  0,  0,  0,  0],
        [19,  5, 20, 36,  7, 18,  5,  5, 14, 36,  9, 14, 36,  2, 36, 19,  9, 24,
         36,  1,  7,  1,  9, 14, 36,  0,  0,  0,  0,  0,  0],
        [12,  1, 25, 36, 23,  8,  9, 20,  5, 36, 23,  9, 20,  8, 36, 26, 36, 19,
          5, 22,  5, 14, 36, 16, 12,  5,  1, 19,  5, 36,  0],
        [12,  1, 25, 36,  7, 18,  5,  5, 14, 36,  1, 20, 36,  6, 36, 20, 23, 15,
         36, 19, 15, 15, 14, 36,  0,  0,  0,  0,  0,  0,  0],
        [12,  1, 25, 36,  2, 12, 21,  5, 36,  9, 14, 36, 24, 36,  6, 15, 21, 18,
         36,  1,  7,  1,  9, 14, 36,  0,  0,  0,  0,  0,  0],

In [23]:
import torch
import numpy as np

def train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=100):
    model.to(device)
    for epoch in range(num_epochs):
        total_loss = 0
        model.train()
        for batch_idx, (inputs, input_lengths, targets, target_lengths) in enumerate(train_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.permute(1, 0, 2)
            outputs = F.log_softmax(outputs, dim=2)
            print(outputs.shape, targets.shape, input_lengths.shape, target_lengths.shape)
            loss = criterion(outputs, targets, input_lengths, target_lengths)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for inputs, input_lengths, targets, target_lengths in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                val_loss = criterion(outputs, targets, input_lengths, target_lengths)
                total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(test_loader)

        print(f'Epoch {epoch+1}, Training Loss: {avg_loss}, Validation Loss: {avg_val_loss}')


vocab = create_vocab()
vocab_len = len(vocab)
num_classes = vocab_len + 1 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = LipReadingRNN(input_size=120, hidden_size=256, num_classes=num_classes).to(device)
criterion = nn.CTCLoss(blank=num_classes - 1, zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=100)


torch.Size([75, 32, 37]) torch.Size([32, 30]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 32]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 34]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 34]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 32]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 30]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 30]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 30]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 30]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 31]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 31]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32, 31]) torch.Size([32]) torch.Size([32])
torch.Size([75, 32, 37]) torch.Size([32,

RuntimeError: input_lengths must be of size batch_size

In [48]:
first_batch = next(iter(test_loader))
    
first_input, _, labels, _ = first_batch 

print("Shape of the input sample for inference:", first_input.shape)

Shape of the input sample for inference: torch.Size([32, 75, 120])


In [74]:
first_input[0].shape

torch.Size([75, 120])

In [76]:
with torch.no_grad():
    log_probs = model(first_input.to(device))


In [50]:
log_probs.shape

torch.Size([75, 32, 37])

In [90]:
log_probs[:,25,:].shape

torch.Size([75, 37])

In [91]:
''.join(int_to_char(idx) for idx in labels[25])

'place white in p three please '

In [92]:
max_indices = [np.argmax(log_prob) for log_prob in torch.exp(log_probs[:,25].cpu())]
''.join(int_to_char(idx) for idx in max_indices)

'set    bre e     it         s     sine            sn          on           '

In [150]:
import torch
import itertools

def greedy_decoder(ctc_output, blank_label=0):
    probabilities = torch.exp(ctc_output)
    _, max_indices = torch.max(probabilities, 2)
    indices = max_indices.squeeze(1)  
    decoded_sequence = [index for index, group in itertools.groupby(indices) if index != blank_label]
    return decoded_sequence

decoded_sequence = greedy_decoder(log_probs)
print("Decoded sequence indices:", decoded_sequence)


Decoded sequence indices: [tensor(12, device='cuda:0'), tensor(1, device='cuda:0'), tensor(25, device='cuda:0'), tensor(36, device='cuda:0'), tensor(18, device='cuda:0'), tensor(5, device='cuda:0'), tensor(36, device='cuda:0'), tensor(5, device='cuda:0'), tensor(36, device='cuda:0'), tensor(9, device='cuda:0'), tensor(20, device='cuda:0'), tensor(36, device='cuda:0'), tensor(5, device='cuda:0'), tensor(36, device='cuda:0')]


In [151]:
''.join(int_to_char(idx) for idx in decoded_sequence)

'lay re e it e '